In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import optuna

In [2]:
from preprocessing import *
data_path = "data/game.csv"
X_train, X_test, y_train, y_test = prep_all(data_path)

X_train = X_train.values.reshape(-1, 1, 135)  # Reshape to (32520, 1, 136)
X_test = X_test.values.reshape(-1, 1, 135)    # Reshape to (N, 1, 136), where N is the number of test samples

train_data = TensorDataset(torch.tensor(X_train), torch.tensor(y_train.to_numpy()))
test_data = TensorDataset(torch.tensor(X_test), torch.tensor(y_test.to_numpy()))

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(test_data, batch_size=64, shuffle=False)

In [3]:
class LogisticRegressionModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(LogisticRegressionModel, self).__init__()
        self.linear = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.linear(x)
        return out


In [4]:
input_size = X_train.shape[2]  # The number of features in your preprocessed data
output_size = 2  # Win or loss (binary classification)

model = LogisticRegressionModel(input_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)


In [5]:
num_epochs = 1000

for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.squeeze(1).float(), labels.long()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/10000], Loss: 0.1346
Epoch [2/10000], Loss: 0.1782
Epoch [3/10000], Loss: 0.0907
Epoch [4/10000], Loss: 0.0366
Epoch [5/10000], Loss: 0.0041
Epoch [6/10000], Loss: 0.0371
Epoch [7/10000], Loss: 0.0420
Epoch [8/10000], Loss: 0.0178
Epoch [9/10000], Loss: 0.0473
Epoch [10/10000], Loss: 0.0187
Epoch [11/10000], Loss: 0.1112
Epoch [12/10000], Loss: 0.0143
Epoch [13/10000], Loss: 0.0179
Epoch [14/10000], Loss: 0.0316
Epoch [15/10000], Loss: 0.0465
Epoch [16/10000], Loss: 0.0212
Epoch [17/10000], Loss: 0.0718
Epoch [18/10000], Loss: 0.0354
Epoch [19/10000], Loss: 0.0463
Epoch [20/10000], Loss: 0.0681
Epoch [21/10000], Loss: 0.0022
Epoch [22/10000], Loss: 0.0325
Epoch [23/10000], Loss: 0.0178
Epoch [24/10000], Loss: 0.0340
Epoch [25/10000], Loss: 0.0075
Epoch [26/10000], Loss: 0.0189
Epoch [27/10000], Loss: 0.0663
Epoch [28/10000], Loss: 0.0519
Epoch [29/10000], Loss: 0.1334
Epoch [30/10000], Loss: 0.0593
Epoch [31/10000], Loss: 0.0282
Epoch [32/10000], Loss: 0.0435


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in valid_loader:
        inputs, labels = inputs.squeeze(1).float(), labels.long()
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f'Test Accuracy: {100 * correct / total:.2f}%')
